In [1]:
import numpy as np
import nltk
import os
import string 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from num2words import num2words
from tqdm import tqdm
import math
import heapq
import re
import pandas as pd
from unidecode import unidecode

### Loading Data

In [2]:
my_dir = 'C:\\Users\\shekh\\Desktop\\Shekhar\\IR\\Assignment 2\\stories'
folders = []
files = []
docs = []
titles = []
names = []
titles = []

idtoname = {}
nametoid = {}

folders.append(my_dir)
for folder in os.listdir(my_dir):
    newpath = my_dir +'\\'+ folder
    #print(newpath)
    if os.path.isdir(newpath):
        folders.append(newpath)
        
#print(folders)

for folder in folders:
    #print(folder)
    for file in os.listdir(folder):
        #print(file)
        if(file == "index.html"):
            file = open(folder+'\\'+file, 'r')
            text = file.read().strip()
            file.close()
            names.extend(re.findall('><A HREF="(.*)">', text))
            titles.extend(re.findall(r'<BR><TD> (.*)\n', text))

names = names[2:]

for name in names:
    for folder in folders:
        for file in os.listdir(folder):
            if file == name:
                f = open(folder+ '\\'+file, 'r', errors = 'ignore')
                doc = f.read() 
                docs.append(doc)
                f.close()


In [3]:
ps = PorterStemmer()

In [4]:
        # inputstr = "The 5,biggest countries by the the the the the.0.7..   population °°°°°°°°°°°°°°°°°°°°°°°°°°°°°furri in 2017 are China, India, time-travel, United States, Indonesia, and Brazil.[!#$%&""()*+,-./:;<=>?@[\]^_`{|}~]:"
        # inputstr2 = "the Box A contains 3 red and 5 American     white balls, while Box B contains 4 red and 2 blue balls.[!#$%&""()*+,-./:;<=>?@[\]^_`{|}~]:"
        # inputstr3 = "The online encyclopedia project brazil Wikipedia is the most popular united states wiki-based website, and is one of the most widely viewed sites in the world, having been ranked in the top ten since 2007."
        # inputstr4 = "the Casey Owen Neistat is an American 5 YouTube personality, india india filmmaker, vlogger and co-founder of the multimedia company Beme, which was later acquired by CNN. In 2018, he founded 368, a creative space for creators to collaborate and influence each other."
        # inputs = []
        # inputs.append(inputstr)
        # inputs.append(inputstr2)
        # inputs.append(inputstr3)
        # inputs.append(inputstr4)
        #print(type(inputs))

In [5]:
print(len(titles))
print(len(docs))

467
467


In [6]:
def listtodict(keys, values):
    keys = keys.copy()
    values = values.copy()
    dic = {}
    for key in range(len(keys)): 
        for value in values: 
            dic[key] = value
            values.remove(value)
            break  
    return dic

In [7]:
def apply_proter_stemmer(string):
    for i in range(len(string)):
        string[i] = ps.stem(string[i])
    return string
        

In [8]:
def remove_stopword(string):
    stop_words = set(stopwords.words('english'))
    data = [w for w in string if not w in stop_words]
    return data

In [9]:
def convert_num(string):
    for i in range(len(string)):
        try:
            if(string[i].isnumeric()):
                string[i] = num2words(string[i])
        except:
            continue
    return string

In [10]:
def preprocessing(newdataset):
    dataset = newdataset.copy()
    for i in tqdm(range(len(dataset))):
        dataset[i] = dataset[i].lower()
        dataset[i] = remove_nonascii(dataset[i])
        dataset[i] = dataset[i].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
        dataset[i] = dataset[i].split()
        dataset[i] = remove_stopword(dataset[i])
        dataset[i] = convert_num(dataset[i])
        dataset[i] = apply_proter_stemmer(dataset[i])
    return dataset

In [11]:
def querypreprocessing(stringdata):
    lower = stringdata.lower()
    punc = lower.translate(str.maketrans('','',string.punctuation))
    spl = punc.split()
    removestop = remove_stopword(spl)
    removenum = convert_num(removestop)
    stemmedquery = apply_proter_stemmer(removenum)
    return stemmedquery

In [12]:
def remove_nonascii(string):
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

In [13]:
#d = preprocessing(inputs)
#print(d)

### Q1 Jaccard Coefficient

In [14]:
def jaccardcoefficient(data, query):
    dataset = set(data)
    queryset = set(query)
    
    unionset = dataset|queryset
    intersectionset = dataset&queryset

    return len(intersectionset)/len(unionset) 

In [15]:
def create_vocabulary(data):
    vocab = set()
    for doc in data:
        for word in doc:
            #print(word)
            vocab.add(word)
    return list(vocab)

In [16]:
#processedsample = preprocessing(inputs)

In [17]:
processedata = preprocessing(docs)

100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [00:29<00:00, 15.96it/s]


In [18]:
vocabulary = create_vocabulary(processedata)

### Jacard Retrieval

In [19]:
def jaccardretrieval(dataset, query,k):
    processedquery = querypreprocessing(query)
    
    for word in processedquery:
        if word not in vocabulary:
            processedquery.remove(word)
    
    jaccard_coeff_list = []
    for data in dataset:
        jaccard_coeff_list.append(jaccardcoefficient(data, processedquery))
    return heapq.nlargest(k, range(len(jaccard_coeff_list)), jaccard_coeff_list.__getitem__), heapq.nlargest(10,jaccard_coeff_list)

In [20]:
for i in range(len(titles)):
    titles[i] = querypreprocessing(titles[i])

In [21]:
idtotitle = listtodict(docs, titles)

In [22]:
idtoname = listtodict(docs, names)

### Enter Query for Question 1

In [50]:
query = input("Enter query: ")
k = int(input("Enter Value of K: "))
result, jaccardcoeff = jaccardretrieval(processedata, query,k)

for (i,j) in zip(result, jaccardcoeff):
    print("ID: ",i," NAME: ",idtoname.get(i)," JACCARD COEFF:", j)
    

Enter query: 100 west by 53 north
Enter Value of K: 10
ID:  317  NAME:  peace.fun  JACCARD COEFF: 0.02040816326530612
ID:  325  NAME:  poem-2.txt  JACCARD COEFF: 0.008333333333333333
ID:  384  NAME:  snowmaid.txt  JACCARD COEFF: 0.008264462809917356
ID:  438  NAME:  weaver.txt  JACCARD COEFF: 0.0078125
ID:  436  NAME:  wall.art  JACCARD COEFF: 0.007692307692307693
ID:  330  NAME:  prince.art  JACCARD COEFF: 0.0058823529411764705
ID:  381  NAME:  sis  JACCARD COEFF: 0.0057251908396946565
ID:  105  NAME:  campfire.txt  JACCARD COEFF: 0.00546448087431694
ID:  387  NAME:  socialvikings.txt  JACCARD COEFF: 0.005333333333333333
ID:  386  NAME:  social.vikings  JACCARD COEFF: 0.005319148936170213


In [24]:
#print(len(vocabulary))

In [25]:
def tf1(data, word):
    return data.count(word)/len(data)

In [26]:
def tf2(data, word):
    if data.count(word) == 0:
        return 0
    else:
        return 1+(math.log10(data.count(word)))

In [27]:
def create_dataframe(vocab, data, termfreq):
    df = pd.DataFrame()
    df['vocab'] = vocab
    
    
    tfall = []
    
    for i in tqdm(range(len(data))):
        tf1 = []
        for word in vocab:
            tf1.append(termfreq(data[i],word))
        tfall.append(tf1)
    #print(tfall)
    
    for i in range(len(tfall)):
        df.insert(i+1, i, tfall[i])
    
    df.set_index('vocab', inplace = True)
    
    nonzero = np.count_nonzero(df,axis = 1) 
    
    idf = []
    for i in tqdm(range(len(nonzero))):
        #print(nonzero[i])
        idf.append(1/(1+(math.log10(df.shape[1]/nonzero[i]))))
    
    df['idf'] = idf
    return df

In [28]:
def create_tfidf(df1):
    df = df1.copy()
    df.iloc[:, 0:-1] = df.iloc[:, 0:-1].mul(df.iloc[:, -1], axis=0)
    #print(df)
    return df

### Q2 Matching Score Retrieval

In [29]:
def matching_score_retrieval(df1, query):
    
    df = df1.copy()
    processedquery = querypreprocessing(query)
    
    for word in processedquery:
        if word not in vocabulary:
            processedquery.remove(word)
    
    df = df.drop(labels = 'idf', axis = 1)
    ds = df.loc[processedquery[0]]
    
    for i in range(1,len(processedquery)):
        ds = ds+df.loc[processedquery[i]]
    
    return ds.sort_values(ascending=False)

### unweighted Dataframe with tf = tf / #of words in doc

In [30]:
df = create_dataframe(vocabulary, processedata, tf1)

100%|████████████████████████████████████████████████████████████████████████| 33137/33137 [00:00<00:00, 244343.78it/s]


### unweighted Dataframe with tf = 1+log(tf) 

In [31]:
df2 = create_dataframe(vocabulary, processedata, tf2)

100%|████████████████████████████████████████████████████████████████████████| 33137/33137 [00:00<00:00, 474648.77it/s]


In [32]:
tfidfdataframe = create_tfidf(df)
tfidfdataframe2 = create_tfidf(df2)

### Enter query for Q2 unweighted tf = tf / #of words in doc

In [51]:
query = input("Enter Query:")
k = int(input("Enter value of k: "))
ser = matching_score_retrieval(tfidfdataframe, query)
retdocs = ser.index.tolist()
#print(retdocs)
#print(idtoname.get(435))
for i in range(0,k):
    print(idtoname.get(retdocs[i]))

Enter Query:50000 variety of flowers
Enter value of k: 10
ghost
narciss.txt
day.in.mcdonald
mcdonaldl.txt
quarter.c9
wall.art
bulnoopt.txt
lgoldbrd.txt
ccm.txt
fantas.hum


### Enter query for Q2 unweighted tf = 1+log(tf)

In [34]:
query = input("Enter Query:")
k = int(input("Enter value of k: "))

ser1 = matching_score_retrieval(tfidfdataframe2, query)
retdocs1 = ser1.index.tolist()
#print(retdocs)
#print(idtoname.get(435))
for i in range(0,k):
    print(idtoname.get(retdocs1[i]))

Enter Query:do cindy abyss
Enter value of k: 10
descent.poe
abyss.txt
sick-kid.txt
empty.txt
breaks3.asc
forgotte
angry_ca.txt
robotech
enchdup.hum
timem.hac


In [35]:
#print("west" in idtotitle[0])

In [36]:
#print(processedata[1].count("get"))

In [37]:
#print(df.shape[0])

In [38]:
def create_weighted_dataframe(vocab, data, heading, termfreq):
    df = pd.DataFrame()
    df['vocab'] = vocab
    titles = heading.copy()
    
    
    tfall = []
    
    for i in tqdm(range(len(data))):
        tf1 = []
        for word in vocab:
            #print(word)
            if(word in idtotitle[i]):
                #print(word, i, idtotitle[i])
                tf1.append(termfreq(data[i],word)*0.7)
            else:
                tf1.append(termfreq(data[i],word)*0.3)
        tfall.append(tf1)
    #print(tfall)
    for i in range(len(tfall)):
        df.insert(i+1, i, tfall[i])
    
    df.set_index('vocab', inplace = True)
    
    nonzero = np.count_nonzero(df,axis = 1) 
    
    idf = []
    for i in tqdm(range(len(nonzero))):
        #print(nonzero[i])
        idf.append(1/(1+(math.log10(df.shape[1]/nonzero[i]))))
    
    df['idf'] = idf
    return df

### weighted Dataframe with tf = tf / #of words in doc

In [39]:
weighteddf = create_weighted_dataframe(vocabulary,processedata,titles, tf1)

100%|████████████████████████████████████████████████████████████████████████| 33137/33137 [00:00<00:00, 461208.79it/s]


### weighted Dataframe with tf = 1+log(tf)

In [40]:
weighteddf2 = create_weighted_dataframe(vocabulary, processedata, titles, tf2)

100%|████████████████████████████████████████████████████████████████████████| 33137/33137 [00:00<00:00, 481535.84it/s]


In [41]:
weightedtfidfdataframe = create_tfidf(weighteddf)
weightedtfidfdataframe2 = create_tfidf(weighteddf2)

### Enter query for Q2 weighted with tf = tf / #of words in doc

In [42]:
query = input("Enter Query:")
k = int(input("Enter value of k: "))

ser2 = matching_score_retrieval(weightedtfidfdataframe, query)
retdocs2 = ser2.index.tolist()
for i in range(0,k):
    print(idtoname.get(retdocs2[i]))

Enter Query:do cindy vampyre
Enter value of k: 10
vampword.txt
peace.fun
forgotte
eyeargon.hum
emperor3.txt
empnclot.txt
empsjowk.txt
empty.txt
enc
encamp01.txt


### Enter query for Q2 weighted with tf =  1+log(tf)

In [58]:
query = input("Enter Query:")
k = int(input("Enter value of k: "))

ser3 = matching_score_retrieval(weightedtfidfdataframe2, query)
retdocs3 = ser3.index.tolist()
for i in range(0,k):
    print(idtoname.get(retdocs3[i]))

Enter Query:do cindy abyss
Enter value of k: 10
abyss.txt
descent.poe
sick-kid.txt
empty.txt
breaks3.asc
forgotte
angry_ca.txt
robotech
enchdup.hum
timem.hac


### Q3 Cosine Retrieval

In [44]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2)/(math.sqrt(np.sum(np.square(vec1)))*math.sqrt(np.sum(np.square(vec2))))

In [45]:
def cosine_retrieval(df1, query):
    
    df = df1.copy()
    
    processedquery = querypreprocessing(query)
    
    for word in processedquery:
        if word not in vocabulary:
            processedquery.remove(word)
    
    #print(processedquery.count("cindi")/len(processedquery))
    queryvector = np.zeros((df.shape[0]))
    #print(len(queryvector))
    wordvector = df.index.values 
    
    for i in range(len(wordvector)):
        for word in processedquery:
            if wordvector[i] == word:
                #print(wordvector[i], word)
                #print(processedquery.count(word)/len(processedquery))
                queryvector[i] = processedquery.count(word)/len(processedquery)
                #print(queryvector[i],i)
        
    
    idf = df.iloc[:,-1].values 
    #print(len(idf))
    #print(queryvector)
    querytfidf = np.multiply(queryvector,idf)
    
    #for i in querytfidf:
     #   if i > 0:
      #      print("queryvector",i)
    
    df = df.drop(labels = 'idf', axis = 1)
    #print(df.shape[1])
    #print(len(queryvector))
    similarity = []
    
    for i in range(df.shape[1]):
        val = tfidfdataframe.iloc[:,i].values
        similarity.append(cosine_similarity(querytfidf, val))
    
    nonzero = []
    
    for i in similarity:
        if i > 0:
            nonzero.append(i)
    #print(nonzero)
    nonzero.sort(reverse=True)
    result = []
    for i in nonzero:
        result.append(similarity.index(i))
        
    return result

### Enter query for Q3 unweighted tf = tf / #of words in doc

In [62]:
query = input("Enter Query:")
k = int(input("Enter value of K:"))
result_cosine = cosine_retrieval(tfidfdataframe, query)
for i in range(int(k)):
    
    print(idtoname[result_cosine[i]])

Enter Query:100 animals
Enter value of K:10
monkking.txt
redragon.txt
hotline4.txt
bran
dwar
horsdonk.txt
ccm.txt
fic4
clevdonk.txt
quarter.c15


### Enter query for Q2 unweighted tf = 1+log(tf)

In [65]:
query = input("Enter Query: ")
k = int(input("Enter Value of k: "))
result_cosine2 = cosine_retrieval(tfidfdataframe2, query)
for i in range(len(result_cosine2)):
    if i == k:
        break
    print(idtoname[result_cosine2[i]])

Enter Query: 100 west by 53 north
Enter Value of k: 10
100west.txt
arctic.txt
bruce-p.txt
peace.fun
candle.hum
dakota.txt
history5.txt
charlie.txt
melissa.txt
sis


### Enter query for Q3 weighted tf = tf / #of words in doc

In [63]:
query = input("Enter Query: ")
k = int(input("Enter Value of k"))

result_weighted_cosine = cosine_retrieval(weightedtfidfdataframe, query)
for i in range(len(result_weighted_cosine)):
    if i == k:
        break
    print(idtoname[result_weighted_cosine[i]])

Enter Query: 100 animals
Enter Value of k10
monkking.txt
redragon.txt
hotline4.txt
bran
dwar
horsdonk.txt
ccm.txt
fic4
clevdonk.txt
quarter.c15


### Enter query for Q3 weighted tf = 1+log(tf)

In [64]:
query = input("Enter Query: ")
k = int(input("Enter Value of k"))

result_weighted_cosine2 = cosine_retrieval(weightedtfidfdataframe2, query)
for i in range(len(result_weighted_cosine2)):
    if i == k:
        break
    print(idtoname[result_weighted_cosine2[i]])

Enter Query: 100 west by 53 north
Enter Value of k10
100west.txt
arctic.txt
bruce-p.txt
peace.fun
candle.hum
dakota.txt
history5.txt
charlie.txt
melissa.txt
sis
